In [ ]:
import pickle
import queue
import datetime
import dateutil
from time import sleep
from dateutil.relativedelta import relativedelta
from kafka import KafkaProducer
from json import dumps

In [ ]:
class custom_clock():
    import datetime
    
    def __init__(self, starttime = datetime.datetime.now()):
        self.invocation_time = datetime.datetime.now()
        self.starttime = starttime
    
    def get_time(self):
        elapsed = datetime.datetime.now() - self.invocation_timeW
        return(self.starttime + elapsed)

In [ ]:
handle = open('data/departures.pickle', 'rb')
departures = pickle.load(handle)

In [ ]:
q = queue.Queue()

In [ ]:
for i in departures:
    q.put(i)


In [ ]:
q.qsize()

In [ ]:
d = datetime.datetime.strptime('20200101 06:15:56', '%Y%m%d %H:%M:%S')

str(d)

In [ ]:

producer = KafkaProducer(
   value_serializer=lambda m: dumps(m).encode('utf-8'), 
   bootstrap_servers=['kafka:9093'])


DATA_YEAR = 2015

clock = custom_clock(datetime.datetime.strptime('20200101 06:15:56', '%Y%m%d %H:%M:%S'))
years_to_add = clock.get_time().year - DATA_YEAR
years_delta = dateutil.relativedelta.relativedelta(years=years_to_add)

while not q.empty():
    departure = q.get()
    departure['departure_dt'] = departure['departure_dt'] + years_delta
    wait =  (departure['departure_dt'] - clock.get_time()).total_seconds()
    departure['departure_dt'] = str(departure['departure_dt'])
    keybite = bytes(departure['id'], encoding='utf-8')
    if wait>=0:
        try:
            print(departure)
            sleep(wait)
            producer.send("departures", key=keybite, value=departure)
            producer.flush()
            
        except:
            pass
            